In [1]:
import os
import torch
import torchvision
import numpy as np
import torch.nn as nn
import scipy.io as sio
import skimage.measure as measure
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.autograd import Variable


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
use_cuda = True
num_steps = 25001
save_frequency = 250
output_name = 'HSI_output/output'
sigma = 1./30

In [2]:
# definations of models 
class AENet(nn.Module):
    def __init__(self):
        super(AENet, self).__init__()
        self.d_conv_1 = nn.Conv2d(31, 8, 5, stride=2, padding=2)
        self.d_bn_1 = nn.BatchNorm2d(8)

        self.d_conv_2 = nn.Conv2d(8, 16, 5, stride=2, padding=2)
        self.d_bn_2 = nn.BatchNorm2d(16)

        self.d_conv_3 = nn.Conv2d(16, 32, 5, stride=2, padding=2)
        self.d_bn_3 = nn.BatchNorm2d(32)
        self.s_conv_3 = nn.Conv2d(32, 4, 5, stride=1, padding=2)

        self.d_conv_4 = nn.Conv2d(32, 64, 5, stride=2, padding=2)
        self.d_bn_4 = nn.BatchNorm2d(64)
        self.s_conv_4 = nn.Conv2d(64, 4, 5, stride=1, padding=2)

        self.d_conv_5 = nn.Conv2d(64, 128, 5, stride=2, padding=2)
        self.d_bn_5 = nn.BatchNorm2d(128)
        self.s_conv_5 = nn.Conv2d(128, 4, 5, stride=1, padding=2)

        self.d_conv_6 = nn.Conv2d(128, 256, 5, stride=2, padding=2)
        self.d_bn_6 = nn.BatchNorm2d(256)

        self.u_deconv_5 = nn.ConvTranspose2d(256, 124, 4, stride=2, padding=1)
        self.u_bn_5 = nn.BatchNorm2d(128)

        self.u_deconv_4 = nn.ConvTranspose2d(128, 60, 4, stride=2, padding=1)
        self.u_bn_4 = nn.BatchNorm2d(64)

        self.u_deconv_3 = nn.ConvTranspose2d(64, 28, 4, stride=2, padding=1)
        self.u_bn_3 = nn.BatchNorm2d(32)

        self.u_deconv_2 = nn.ConvTranspose2d(32, 16, 4, stride=2, padding=1)
        self.u_bn_2 = nn.BatchNorm2d(16)

        self.u_deconv_2 = nn.ConvTranspose2d(32, 16, 4, stride=2, padding=1)
        self.u_bn_2 = nn.BatchNorm2d(16)

        self.u_deconv_1 = nn.ConvTranspose2d(16, 8, 4, stride=2, padding=1)
        self.u_bn_1 = nn.BatchNorm2d(8)

        self.out_deconv = nn.ConvTranspose2d(8, 31, 4, stride=2, padding=1)        
        self.out_bn = nn.BatchNorm2d(31)

        
    def forward(self, noise):
        down_1 = self.d_conv_1(noise)
        down_1 = self.d_bn_1(down_1)
        down_1 = F.leaky_relu(down_1)
        
        down_2 = self.d_conv_2(down_1)
        down_2 = self.d_bn_2(down_2)
        down_2 = F.leaky_relu(down_2)

        down_3 = self.d_conv_3(down_2)
        down_3 = self.d_bn_3(down_3)
        down_3 = F.leaky_relu(down_3)
        skip_3 = self.s_conv_3(down_3)

        down_4 = self.d_conv_4(down_3)
        down_4 = self.d_bn_4(down_4)
        down_4 = F.leaky_relu(down_4)
        skip_4 = self.s_conv_4(down_4)

        down_5 = self.d_conv_5(down_4)
        down_5 = self.d_bn_5(down_5)
        down_5 = F.leaky_relu(down_5)
        skip_5 = self.s_conv_5(down_5)

        down_6 = self.d_conv_6(down_5)
        down_6 = self.d_bn_6(down_6)
        down_6 = F.leaky_relu(down_6)

        up_5 = self.u_deconv_5(down_6)
        up_5 = torch.cat([up_5, skip_5], 1)
        up_5 = self.u_bn_5(up_5)
        up_5 = F.leaky_relu(up_5)

        up_4 = self.u_deconv_4(up_5)
        up_4 = torch.cat([up_4, skip_4], 1)
        up_4 = self.u_bn_4(up_4)
        up_4 = F.leaky_relu(up_4)

        up_3 = self.u_deconv_3(up_4)
        up_3 = torch.cat([up_3, skip_3], 1)
        up_3 = self.u_bn_3(up_3)
        up_3 = F.leaky_relu(up_3)

        up_2 = self.u_deconv_2(up_3)
        up_2 = self.u_bn_2(up_2)
        up_2 = F.leaky_relu(up_2)

        up_1 = self.u_deconv_1(up_2)
        up_1 = self.u_bn_1(up_1)
        up_1 = F.leaky_relu(up_1)

        out = self.out_deconv(up_1)
        out = self.out_bn(out)
        out = F.sigmoid(out)

        return out   

In [3]:
# definations of data processing functions
def load_HSI(path):
    data = sio.loadmat(path)
    im_gt = np.array(data['GT']).astype(np.float32)
    im_h = np.array(data['H']).astype(np.float32)
    im_m = np.array(data['M']).astype(np.float32)
    return im_h, im_m, im_gt

def show_data(img, title=''):
    plt.figure(title)
    plt.imshow(img)
    plt.axis('off')
    plt.title(title)
    plt.show()
    return

def create_P():
    P_ = [[2,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  6, 11, 17, 21, 22, 21, 20, 20, 19, 19, 18, 18, 17, 17],
          [1,  1,  1,  1,  1,  1,  2,  4,  6,  8, 11, 16, 19, 21, 20, 18, 16, 14, 11,  7,  5,  3,  2,  2,  1,  1,  2,  2,  2,  2,  2],
          [7, 10, 15, 19, 25, 29, 30, 29, 27, 22, 16,  9,  2,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]]
    P = np.array(P_, dtype=np.float32)
    div_t = np.sum(P, axis=1)
    for i in range(3):
        P[i,] = P[i,]/div_t[i]
    return P

def downsample(Z):
    ZH = Z[:, :, ::4, ::4]
    return ZH

def PSNR(Z, GT):
    im1 = np.maximum(np.minimum(Z, 1.0), 0.0)
    im2 = np.maximum(np.minimum(GT, 1.0), 0.0)
    return measure.compare_psnr(im1, im2)

def PSNR_GPU(im_true, im_fake):
    data_range = 1
    C = im_true.size()[0]
    H = im_true.size()[1]
    W = im_true.size()[2]
    Itrue = im_true.clone().resize_(C*H*W)
    Ifake = im_fake.clone().resize_(C*H*W)
    #mse = nn.MSELoss(reduce=False)
    err = torch.pow(Itrue-Ifake,2).sum(dim=0, keepdim=True).div_(C*H*W)
    psnr = 10. * torch.log((data_range**2)/err) / np.log(10.)
    return torch.mean(psnr)

def SSIM(Z, GT):
    im1 = np.maximum(np.minimum(Z, 1.0), 0.0)
    im2 = np.maximum(np.minimum(GT, 1.0), 0.0)
    return measure.compare_ssim(im1, im2, win_size=11, data_range=1, gaussian_weights=True)

def SAM_GPU(im_true, im_fake):
    C = im_true.size()[0]
    H = im_true.size()[1]
    W = im_true.size()[2]
    Itrue = im_true.clone().resize_(C, H*W)
    Ifake = im_fake.clone().resize_(C, H*W)
    nom = torch.mul(Itrue, Ifake).sum(dim=0).resize_(H*W)
    denom1 = torch.pow(Itrue,2).sum(dim=0).sqrt_().resize_(H*W)
    denom2 = torch.pow(Ifake,2).sum(dim=0).sqrt_().resize_(H*W)
    sam = torch.div(nom, torch.mul(denom1, denom2)).acos_().resize_(H*W)
    sam = sam / np.pi * 180
    sam = torch.sum(sam) / (H*W)
    return sam

def save_result(Z, outputname):
    sio.savemat(outputname, {'result': Z})
    return
    

In [ ]:
# main processing step
if __name__ == '__main__':
    # load 
    print('load data...')
    datapath = './HSI_dataset/Harvard/imge5.mat'
    im_h, im_m, im_gt = load_HSI(datapath)
    P = create_P()
    if use_cuda:
        P_cu = Variable(torch.from_numpy(P.copy()), requires_grad=False).cuda()
        im_h = Variable(torch.from_numpy(im_h.copy()), requires_grad=False).cuda()
        im_m = Variable(torch.from_numpy(im_m.copy()), requires_grad=False).cuda()
        im_gt = Variable(torch.from_numpy(im_gt.copy()), requires_grad=False).cuda()
    else:
        P_cu = Variable(torch.from_numpy(P.copy()), requires_grad=False)
        im_h = Variable(torch.from_numpy(im_h.copy()), requires_grad=False)
        im_m = Variable(torch.from_numpy(im_m.copy()), requires_grad=False)
        im_gt = Variable(torch.from_numpy(im_gt.copy()), requires_grad=False)
    
    shape_gt = im_gt.shape
    print('initialize net...')
    if use_cuda:
        noise = Variable(torch.randn(shape_gt).cuda()).view(1,
                                        shape_gt[0], shape_gt[1], shape_gt[2])
    else:
        noise = Variable(torch.randn(shape_gt)).view(1,
                                        shape_gt[0], shape_gt[1], shape_gt[2])
    net = AENet()
    if use_cuda:
        net.cuda()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
    
    print('optimize...')
    save_img_ind = 0
    psnrs = []
    sams = []
    losses = []
    steps = []
    for step in range(num_steps):
        output = net(noise)
        # print('At step {},\toutput...'.format(step))
        # degrade
        shape = output.shape
        out = output.view(shape[1], shape[2]*shape[3])
        PZ = torch.matmul(P_cu, out).view(im_m.shape[0], shape[2],shape[3])
        # print('At step {},\tPZ output...'.format(step))
        ZH = torch.squeeze(downsample(output))
        optimizer.zero_grad()
        loss = torch.sum((ZH - im_h)**2) + torch.sum((PZ - im_m)**2)
        # print('At step {},\tloss...'.format(step))
        loss.backward()
        # print('At step {},\tbackward...'.format(step))
        optimizer.step()
        psnrs.append(PSNR_GPU(im_gt.data.cpu(), torch.squeeze(output).data.cpu()))
        sams.append(SAM_GPU(im_gt.data.cpu(), torch.squeeze(output).data.cpu()))
        losses.append(loss.data[0])
        steps.append(step)
        print('step: {} loss: {} psnr: {} sam:{}'.format(step, losses[step],
                                psnrs[step], sams[step]))
        if step % save_frequency == 0:
            save_result(torch.squeeze(output).data.cpu().numpy(),output_name+'_{}.mat'.format(save_img_ind))
            save_img_ind += 1
            sio.savemat('loss_psnr_sam_results.mat', {
                'steps': steps, 'losses': losses, 'psnrs':psnrs, 'sams':sams})
        if use_cuda:
            noise.data += sigma * torch.randn(noise.shape).cuda()
        else:
            noise.data += sigma * torch.randn(noise.shape)
            
    if use_cuda:
        torch.cuda.empty_cache()

load data...
initialize net...
optimize...
step: 0 loss: 839097.0625 psnr: 7.70053768157959 sam:30.659395338084323
step: 1 loss: 723974.625 psnr: 8.163339614868164 sam:29.504274019136574
step: 2 loss: 673455.0 psnr: 7.762294769287109 sam:30.182230745293964
step: 3 loss: 647314.9375 psnr: 7.800388813018799 sam:29.930467616926762
step: 4 loss: 627193.375 psnr: 7.884176731109619 sam:29.793092216949844
step: 5 loss: 602655.125 psnr: 8.068741798400879 sam:29.663584930400248
step: 6 loss: 571514.4375 psnr: 8.413406372070312 sam:29.35884270396764
step: 7 loss: 546763.1875 psnr: 8.79085636138916 sam:28.99091200267958
step: 8 loss: 526347.5 psnr: 9.005664825439453 sam:28.86881910138618
step: 9 loss: 504195.375 psnr: 9.064591407775879 sam:28.94618987562353
step: 10 loss: 481638.15625 psnr: 9.147760391235352 sam:28.99755386511697
step: 11 loss: 458703.5 psnr: 9.284557342529297 sam:29.00133643328627
step: 12 loss: 436944.0 psnr: 9.392165184020996 sam:29.029372656930718
step: 13 loss: 416650.125 ps

step: 109 loss: 69862.96875 psnr: 14.997764587402344 sam:30.337856331697765
step: 110 loss: 68961.453125 psnr: 15.028816223144531 sam:30.343320933271457
step: 111 loss: 68057.7421875 psnr: 15.063224792480469 sam:30.342709726553153
step: 112 loss: 67202.28125 psnr: 15.09277057647705 sam:30.355467284908173
step: 113 loss: 66369.8671875 psnr: 15.117743492126465 sam:30.373886798207593
step: 114 loss: 65515.51953125 psnr: 15.153461456298828 sam:30.368474573247
step: 115 loss: 64713.66796875 psnr: 15.188297271728516 sam:30.36335464345393
step: 116 loss: 63904.3203125 psnr: 15.212824821472168 sam:30.38303169792539
step: 117 loss: 63132.4375 psnr: 15.242176055908203 sam:30.391692223279847
step: 118 loss: 62338.89453125 psnr: 15.276198387145996 sam:30.387574497350386
step: 119 loss: 61592.9765625 psnr: 15.303400039672852 sam:30.395227710928964
step: 120 loss: 60826.703125 psnr: 15.336804389953613 sam:30.391729451013873
step: 121 loss: 60077.484375 psnr: 15.368135452270508 sam:30.391636387882045

step: 215 loss: 22365.255859375 psnr: 17.078250885009766 sam:29.069156761568195
step: 216 loss: 22129.05859375 psnr: 17.08075714111328 sam:29.065025259298466
step: 217 loss: 21927.50390625 psnr: 17.092453002929688 sam:29.054360730724625
step: 218 loss: 21758.486328125 psnr: 17.11345672607422 sam:29.017301736448303
step: 219 loss: 21603.806640625 psnr: 17.11393165588379 sam:29.004551071067397
step: 220 loss: 21411.3671875 psnr: 17.13043975830078 sam:28.99880458640962
step: 221 loss: 21177.474609375 psnr: 17.138507843017578 sam:28.984544802629273
step: 222 loss: 21040.0703125 psnr: 17.147296905517578 sam:28.96140061468742
step: 223 loss: 20861.841796875 psnr: 17.16547393798828 sam:28.939958911081874
step: 224 loss: 20669.35546875 psnr: 17.17394256591797 sam:28.92935859444384
step: 225 loss: 20522.5703125 psnr: 17.180091857910156 sam:28.91654053136267
step: 226 loss: 20365.32421875 psnr: 17.19281578063965 sam:28.89696602439261
step: 227 loss: 20178.634765625 psnr: 17.200138092041016 sam:2

step: 320 loss: 10733.5107421875 psnr: 17.869876861572266 sam:27.331028297660595
step: 321 loss: 10677.44921875 psnr: 17.882732391357422 sam:27.30321183363307
step: 322 loss: 10619.6357421875 psnr: 17.871971130371094 sam:27.302424857636254
step: 323 loss: 10563.53125 psnr: 17.88304328918457 sam:27.29228867775103
step: 324 loss: 10505.2080078125 psnr: 17.88340950012207 sam:27.28728617871883
step: 325 loss: 10451.9658203125 psnr: 17.888952255249023 sam:27.259780331251477
step: 326 loss: 10387.94921875 psnr: 17.89042854309082 sam:27.246590352467024
step: 327 loss: 10336.984375 psnr: 17.88863754272461 sam:27.24278912903287
step: 328 loss: 10271.0673828125 psnr: 17.90381622314453 sam:27.210053167840385
step: 329 loss: 10204.6171875 psnr: 17.908872604370117 sam:27.181201662100023
step: 330 loss: 10169.5361328125 psnr: 17.9105224609375 sam:27.165745736342643
step: 331 loss: 10098.697265625 psnr: 17.918968200683594 sam:27.13964562270496
step: 332 loss: 10044.8955078125 psnr: 17.918243408203125

step: 425 loss: 6565.263671875 psnr: 18.18196678161621 sam:25.59126119209037
step: 426 loss: 6530.28564453125 psnr: 18.199478149414062 sam:25.560078940307335
step: 427 loss: 6528.53369140625 psnr: 18.189607620239258 sam:25.553530117053413
step: 428 loss: 6485.7939453125 psnr: 18.196800231933594 sam:25.536773270188064
step: 429 loss: 6436.24951171875 psnr: 18.203144073486328 sam:25.501985053733392
step: 430 loss: 6425.00927734375 psnr: 18.20259666442871 sam:25.488694001975716
step: 431 loss: 6411.88037109375 psnr: 18.206565856933594 sam:25.485126816663126
step: 432 loss: 6364.38623046875 psnr: 18.206388473510742 sam:25.452089491404593
step: 433 loss: 6334.435546875 psnr: 18.209089279174805 sam:25.443315414443276
step: 434 loss: 6342.6806640625 psnr: 18.21099090576172 sam:25.450954553757242
step: 435 loss: 6301.16064453125 psnr: 18.214405059814453 sam:25.4047686763422
step: 436 loss: 6258.36865234375 psnr: 18.217256546020508 sam:25.387748692505
step: 437 loss: 6272.5869140625 psnr: 18.21

...

step: 11056 loss: 361.3390197753906 psnr: 16.455501556396484 sam:26.23302898052623
step: 11057 loss: 361.8534240722656 psnr: 16.460163116455078 sam:26.222796380707138
step: 11058 loss: 363.1517333984375 psnr: 16.457157135009766 sam:26.22550615946517
step: 11059 loss: 364.6526794433594 psnr: 16.45806884765625 sam:26.22835958842407
step: 11060 loss: 368.2802734375 psnr: 16.4583683013916 sam:26.22029310925967
step: 11061 loss: 371.7671813964844 psnr: 16.452783584594727 sam:26.251812251662102
step: 11062 loss: 376.89715576171875 psnr: 16.457542419433594 sam:26.221270824022298
step: 11063 loss: 379.6845397949219 psnr: 16.452421188354492 sam:26.2514709750331
step: 11064 loss: 383.1943359375 psnr: 16.459815979003906 sam:26.211622833113125
step: 11065 loss: 381.74786376953125 psnr: 16.449125289916992 sam:26.271457806767614
step: 11066 loss: 378.6763916015625 psnr: 16.45863151550293 sam:26.219072956326556
step: 11067 loss: 372.99371337890625 psnr: 16.446840286254883 sam:26.281116880582914
step:

step: 11155 loss: 363.8588562011719 psnr: 16.440963745117188 sam:26.287318076205082
step: 11156 loss: 364.6533203125 psnr: 16.43414878845215 sam:26.31604716676742
step: 11157 loss: 366.8326721191406 psnr: 16.44532585144043 sam:26.272045558442613
step: 11158 loss: 368.45849609375 psnr: 16.43018341064453 sam:26.335064049684348
step: 11159 loss: 371.82904052734375 psnr: 16.447839736938477 sam:26.2636315094893
step: 11160 loss: 374.122802734375 psnr: 16.42427635192871 sam:26.361289208535624
step: 11161 loss: 379.0018005371094 psnr: 16.449825286865234 sam:26.256135096017914
step: 11162 loss: 382.13287353515625 psnr: 16.42284393310547 sam:26.365969562136797
step: 11163 loss: 389.61517333984375 psnr: 16.451148986816406 sam:26.246509318496237
step: 11164 loss: 392.2473449707031 psnr: 16.421884536743164 sam:26.366228126809517
step: 11165 loss: 399.1824951171875 psnr: 16.448528289794922 sam:26.252197428448937
step: 11166 loss: 400.1170654296875 psnr: 16.42522621154785 sam:26.348513116286966
step

step: 11254 loss: 360.7322998046875 psnr: 16.429859161376953 sam:26.327675513813688
step: 11255 loss: 363.4222717285156 psnr: 16.435182571411133 sam:26.314294691658972
step: 11256 loss: 367.4420471191406 psnr: 16.42486572265625 sam:26.343878116825863
step: 11257 loss: 375.10308837890625 psnr: 16.43804359436035 sam:26.302257208829474
step: 11258 loss: 384.80352783203125 psnr: 16.422901153564453 sam:26.348882098310384
step: 11259 loss: 402.3209228515625 psnr: 16.441024780273438 sam:26.288497764130625
step: 11260 loss: 410.61029052734375 psnr: 16.412179946899414 sam:26.394857644767626
step: 11261 loss: 427.0877380371094 psnr: 16.44428062438965 sam:26.2745529680154
step: 11262 loss: 414.2013244628906 psnr: 16.403305053710938 sam:26.43881781657396
step: 11263 loss: 414.7528076171875 psnr: 16.444198608398438 sam:26.285142561972236
step: 11264 loss: 403.07440185546875 psnr: 16.403879165649414 sam:26.454251760573612
step: 11265 loss: 407.82696533203125 psnr: 16.43610954284668 sam:26.3140552803

step: 11353 loss: 357.83258056640625 psnr: 16.430416107177734 sam:26.294134090887326
step: 11354 loss: 356.7418212890625 psnr: 16.43210220336914 sam:26.28527580814071
step: 11355 loss: 356.6783142089844 psnr: 16.429677963256836 sam:26.296423647884126
step: 11356 loss: 356.4967346191406 psnr: 16.429641723632812 sam:26.29821694914594
step: 11357 loss: 356.8972473144531 psnr: 16.42990493774414 sam:26.297017285432588
step: 11358 loss: 357.6611022949219 psnr: 16.428821563720703 sam:26.302547612486222
step: 11359 loss: 357.8466491699219 psnr: 16.42945671081543 sam:26.296283366455384
step: 11360 loss: 358.391357421875 psnr: 16.42829132080078 sam:26.306106296784094
step: 11361 loss: 358.4753723144531 psnr: 16.42698860168457 sam:26.306786271854662
step: 11362 loss: 358.444580078125 psnr: 16.426488876342773 sam:26.31215165624033
step: 11363 loss: 358.34173583984375 psnr: 16.428028106689453 sam:26.29964512788638
step: 11364 loss: 357.9273376464844 psnr: 16.426557540893555 sam:26.312298196662653
s

step: 11452 loss: 387.02044677734375 psnr: 16.393714904785156 sam:26.460371349498047
step: 11453 loss: 387.23095703125 psnr: 16.415407180786133 sam:26.370264913269096
step: 11454 loss: 379.95770263671875 psnr: 16.395299911499023 sam:26.454411662279995
step: 11455 loss: 374.27984619140625 psnr: 16.410680770874023 sam:26.401772175108704
step: 11456 loss: 366.86944580078125 psnr: 16.401357650756836 sam:26.439277560212616
step: 11457 loss: 362.9216613769531 psnr: 16.40152931213379 sam:26.445657606732624
step: 11458 loss: 364.86907958984375 psnr: 16.407251358032227 sam:26.416631170614068
step: 11459 loss: 368.7381591796875 psnr: 16.397218704223633 sam:26.46115108692868
step: 11460 loss: 370.77197265625 psnr: 16.410642623901367 sam:26.40470845127868
step: 11461 loss: 366.7825622558594 psnr: 16.396787643432617 sam:26.45603416035391
step: 11462 loss: 362.8089904785156 psnr: 16.408702850341797 sam:26.408045399529442
step: 11463 loss: 361.9723815917969 psnr: 16.398754119873047 sam:26.44540085939

In [51]:
# load data test
datapath = './HSI_dataset/Harvard/imge5.mat'
im_h, im_m, im_gt = load_HSI(datapath)
im_h_1 = np.squeeze(im_h[1,:,:])
im_m_1 = np.squeeze(im_m[1,:,:])
im_gt_1 = np.squeeze(im_gt[1,:,:])
show_data(im_h_1, 'H_1')
show_data(im_m_1, 'M_1')
show_data(im_gt_1, 'GT_1')